In [ ]:
#import openai
#import os

Now to test the key

In [6]:
from llama_cpp import Llama

# Update the model_path with the full (or relative) path to your .llamafile
model_path = "C:/Users/hpfin/OneDrive/Desktop/Time Series/Mines/Junior/LLM/FinalProject/whisper-large-v3.bin"

# Initialize the model; note that the library must support your file format.
llm = Llama(model_path=model_path)

def test_model():
    prompt = "Explain how to integrate a local model in VSCode."
    # Call the model with your prompt and adjust parameters as needed.
    response = llm(prompt, max_tokens=100, temperature=0.7)
    # The format of the response may differ; here we assume it returns a dict with a 'choices' key.
    # For example purposes, we simply print the full response.
    print("Response from the model:")
    print(response)

if __name__ == "__main__":
    test_model()


llama_model_load: error loading model: llama_model_loader: failed to load model from C:/Users/hpfin/OneDrive/Desktop/Time Series/Mines/Junior/LLM/FinalProject/whisper-large-v3.bin

llama_model_load_from_file_impl: failed to load model


ValueError: Failed to load model from file: C:/Users/hpfin/OneDrive/Desktop/Time Series/Mines/Junior/LLM/FinalProject/whisper-large-v3.bin

In [ ]:
import openai
import os
from dotenv import load_dotenv
import unittest
import io
import sys
import time

# Load environment variables from a .env file (if using one)
load_dotenv()

# Set your OpenAI API key. You can also hard-code it here temporarily.
openai.api_key = os.getenv("OPENAI_API_KEY")
# For testing purposes only (do NOT commit or share hard-coded API keys):
# openai.api_key = "sk-YourActualKeyHere"

def test_o3_mini_high() -> str:
    """
    Calls the OpenAI API using the legacy completions endpoint with the 'o3-mini-high' engine,
    using a prompt that describes the assignment to implement a domino chain solver.
    Returns the candidate solution code as a string.
    """
    response = openai.Completion.create(
        engine="o3-mini-high",  # using the legacy 'engine' parameter
        prompt="""# Assignment: Maximum Domino Chain

**Problem Description**
You are given a list of domino pieces, where each domino is represented as a tuple of two integers (e.g., (a, b)). Your task is to write a Python function that finds a domino chain using as many pieces as possible from the input list. A domino chain is defined as a sequence of domino pieces where the second number of one piece matches the first number of the next piece. Domino pieces may be flipped (i.e., (a, b) can be used as (b, a)) to allow for chaining.

**Requirements**
1. Implement a function max_domino_chain(dominoes: List[Tuple[int, int]]) -> List[Tuple[int, int]].
2. The function should return a valid domino chain (as a list of domino pieces) that uses the maximum number of dominoes possible from the input.
3. Each domino piece can be used at most once in the chain.
4. Domino pieces may be flipped if necessary.
5. If multiple chains have the maximum length, you may return any one of them.
6. If no domino pieces are provided or if no valid chain can be formed, return an empty list.
7. Assume the input size will not exceed 15 dominoes.

**Example**
Example input: dominoes = [(1, 2), (3, 1), (2, 3)]
One valid chain (after flipping as needed) is: [(3, 1), (1, 2), (2, 3)]
Explanation: 3 -> 1 connects to 1 -> 2, which connects to 2 -> 3.
""",
        max_tokens=100,
        temperature=0.7
    )
    candidate_code = response.choices[0].text.strip()
    print("Response from o3-mini-high:")
    print(candidate_code)
    return candidate_code

def grade_o3_mini_high(candidate_code: str) -> int:
    """
    Grades the candidate solution code by performing the following:
      1. Writes candidate_code to a file named solution.py.
      2. Loads and runs the testsuite from testsuite.py using unittest.
      3. Returns an integer grade (0 to 50) corresponding to the number of tests passed.
    """
    # Write the candidate solution to solution.py
    with open("solution.py", "w") as f:
        f.write(candidate_code)
    
    # Discover and run the testsuite from testsuite.py.
    loader = unittest.TestLoader()
    # Discover tests in the current directory with filename pattern testsuite.py
    suite = loader.discover('.', pattern='testsuite.py')
    
    # Capture the test output into an in-memory stream (so that it doesn't clutter the console)
    stream = io.StringIO()
    runner = unittest.TextTestRunner(stream=stream, verbosity=2)
    result = runner.run(suite)
    
    # Calculate score: each test passed (i.e. not failing or erroring) is one point.
    tests_run = result.testsRun
    failures = len(result.failures)
    errors = len(result.errors)
    passed = tests_run - failures - errors
    
    # Optionally, print the test output for debugging:
    print("Test output:\n" + stream.getvalue())
    print(f"Passed {passed} out of {tests_run} tests.")
    
    return passed

def main():
    grades = []  # empty list to store grades for each iteration
    iterations = 300
    for i in range(iterations):
        print(f"\n--- Iteration {i+1} ---")
        candidate_code = test_o3_mini_high()
        grade = grade_o3_mini_high(candidate_code)
        grades.append(grade)
        # Optional: Add a sleep to avoid hitting API rate limits.
        time.sleep(1)
    
    print("\nFinal Grades:")
    print(grades)

if __name__ == "__main__":
    main()
